## Ideal solution objects

In [5]:
import numpy as np

import reactord as rd

To generate an ideal solution mixture, first it's necessary to create the Substance objects that will be in the mixture. For instance:

In [36]:
meoh = rd.Substance.from_thermo_database("methanol")
h2o = rd.Substance.from_thermo_database("water")
acetone = rd.Substance.from_thermo_database("acetone")
etoh = rd.Substance.from_thermo_database("ethanol")

Now we create a mixture, introducing the objects as a dictionary. In this case, an instance of the IdealSolution class is created:

In [47]:
ideal_sol_mixture = rd.mix.IdealSolution(A=meoh, B=h2o, C=acetone, D=etoh)

# The mixture is a list of substances.
# We can accsess to the properties of individual substances. For example:
print(ideal_sol_mixture.substances[0].normal_boiling_point)
print(ideal_sol_mixture.substances[1].name)
print(ideal_sol_mixture.substances[2].critical_pressure)
print(ideal_sol_mixture.substances[3].name)

337.65
water
4700000.0
ethanol


The IdealSolution class is an instance of the abstract class AbstractMix. As a consequence, it has methods that are common to all the instances of AbstractMix:

- mol_fractions
- mixture_molecular_weight
- partial_pressures
- molar_density
- mass_density

and IdealSolution specific methods:

- concentrations
- volume
- mix_heat_capacity
- formation_enthalpies_correction
- mixture_viscosity

The methods inherited from AbstractMix are explained in the [AbstractMix tutorial](abstract_mix.ipynb). On the other hand, each method implemented by IdealSolution may require that the substances have specific methods or atributes defined. Let's see the methods.

__concentrations (moles, temperature, pressure)__

This method recieves the moles of each substance, the temperature [K] and pressure [Pa] and returns the concentration of each substance $C_i$ [$moles/m^3$]. The calculation is made by dividing each molar fraction $z_i$ by the total molar volume $V_T$:

$$ C_i = \frac {z_i} {V_T} $$


In [49]:
moles = [1.5, 2, 0.5, 3]
temperature = 298.15  # K
pressure = 101325  # Pa
concentrations = np.round(
    ideal_sol_mixture.concentrations(moles, temperature, pressure), 2
)
print(f"Concentrations (mol/m³): {concentrations}")

Concentrations (mol/m³): [4842.05 6456.06 1614.02 9684.09]


__volume (moles, temperature, pressure)__

This method recieves the moles of each substance, the temperature [K] and pressure [Pa] and returns the volume of the mixture [$m^3$]. The calculation is done by the dot product of the substance's molar volumes $v_i$ and the moles $n_i$:

$$ V = v_i · n_i

In [39]:
molar_volume = ideal_sol_mixture.volume(moles, temperature, pressure)
print(f"Molar volume of the mixture: {molar_volume} m³/mol")

Molar volume of the mixture: 0.00030978644858994757 m³/mol


__mix_heat_capacity (moles, temperature, pressure)__

This method recieves the moles of each substance, the temperature [K] and pressure [Pa] and returns the heat capacity of the mixture [$J/K$]. 
The calculation is performed by the dot product between the molar fractions $z_i$ and the ${c_p}_i$ of the compounds:

$$ {C_p}_{mix} = z_i · {C_p}_i

In [51]:
heat_capacity = np.round(
    ideal_sol_mixture.mix_heat_capacity(moles, temperature, pressure), 3
)

print(f"Mixture heat capacity: {heat_capacity} J/K")

Mixture heat capacity: 95.798 J/K


__formation_enthalpies_correction (moles, temperature, pressure)__

This method recieves the moles of each substance, the temperature [K] and pressure [Pa] and returns a term to correct the formation enthalpies [$J/mol/K$] of the components of the mixture at the given temperature. The formula to calculate the corrected formation enthalpies is:

$${\Delta H_{f}}_{i} = {\Delta H_{i}^{298.15}} + \int_{298.15}^{T}{{C_p}_{i}}{(T,P)}dT$$

this method evaluates and solve the integral term. It also takes into account the heat required to melt a substance, if its boiling point is higher than 298.15 K.

The formation enthalpy at non-standard conditions is calculated by adding the standard formation enthalpy and the correction term.
The standard formation enthalpies are set using the private method _formation_enthalpies_set() as shown below:

In [46]:
standard_enthalpies = ideal_sol_mixture._formation_enthalpies_set()
enthalpy_correction = np.round(
    ideal_sol_mixture.formation_enthalpies_correction(400, pressure), 0
)
corrected_enthalpies = np.round(standard_enthalpies + enthalpy_correction, 0)

print(f"Standard formation enthalpies: {standard_enthalpies} J/(mol.K)")
print(f"Enthalpy correction: {enthalpy_correction} J/(mol.K)")
print(f"Corrected enthalpies: {corrected_enthalpies} J/(mol.K)")

Standard formation enthalpies: [-238400. -285825. -248400. -277030.] J/(mol.K)
Enthalpy correction: [ 9717.  7711. 13639. 14094.] J/(mol.K)
Corrected enthalpies: [-228683. -278114. -234761. -262936.] J/(mol.K)


__mixture_viscosity (moles, temperature, pressure)__

NOT IMPLEMENTED YET